In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor

/Users/mathiasotnes/anaconda3/envs/autogluon/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Import data
obs_A = pd.read_parquet('../../preprocessing/data/obs_A.parquet')
obs_B = pd.read_parquet('../../preprocessing/data/obs_B.parquet')
obs_C = pd.read_parquet('../../preprocessing/data/obs_C.parquet')
est_A = pd.read_parquet('../../preprocessing/data/est_A.parquet')
est_B = pd.read_parquet('../../preprocessing/data/est_B.parquet')
est_C = pd.read_parquet('../../preprocessing/data/est_C.parquet')
test_A = pd.read_parquet('../../preprocessing/data/test_A.parquet')
test_B = pd.read_parquet('../../preprocessing/data/test_B.parquet')
test_C = pd.read_parquet('../../preprocessing/data/test_C.parquet')

In [4]:
# Concatinate
A = pd.concat([obs_A, est_A])
B = pd.concat([obs_B, est_B])
C = pd.concat([obs_C, est_C])

# Data splits for submissions
X_A = A.drop(columns='pv_measurement')
y_A = A['pv_measurement']
X_B = B.drop(columns='pv_measurement')
y_B = B['pv_measurement']
X_C = C.drop(columns='pv_measurement')
y_C = C['pv_measurement']

# Data splits for testing
# train_A, test_A = train_test_split(A, test_size=0.2, shuffle=True, random_state=42)
# X_train_A = train_A.drop(columns='pv_measurement')
# y_train_A = train_A['pv_measurement']
# X_test_A = test_A.drop(columns='pv_measurement')
# y_test_A = test_A['pv_measurement']

# train_B, test_B = train_test_split(B, test_size=0.2, shuffle=True, random_state=42)
# X_train_B = train_B.drop(columns='pv_measurement')
# y_train_B = train_B['pv_measurement']
# X_test_B = test_B.drop(columns='pv_measurement')
# y_test_B = test_B['pv_measurement']

# train_C, test_C = train_test_split(C, test_size=0.2, shuffle=True, random_state=42)
# X_train_C = train_C.drop(columns='pv_measurement')
# y_train_C = train_C['pv_measurement']
# X_test_C = test_C.drop(columns='pv_measurement')
# y_test_C = test_C['pv_measurement']



In [5]:
# Optional AutoGluon preprocessing
# train_data = TabularDataset(obs_A)
# test_data = TabularDataset(est_A)



In [6]:
label = 'pv_measurement'
test_A.describe()

,total_radiation,clear_sky_rad:W,sun_elevation:d,effective_cloud_cover:p,rain_water:kgm2,hour,sun_azimuth:d,total_cloud_cover:p,snow_accumulation,binned_month,super_cooled_liquid_water:kgm2,year,sfc_pressure:hPa_scaled,average_wind_speed,t_1000hPa:C,absolute_humidity_2m:gm3,msl_pressure:hPa_scaled,temp_dewpoint_diff,dew_or_rime:idx
count,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.0,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000
mean,199.684021,341.014008,20.294441,64.132294,0.013333,11.500000,179.181381,69.309792,0.001563,0.066667,0.063924,2023.0,0.704526,2.407205,11.573374,8.205903,0.707816,3.923195,0.034722
std,215.015656,306.911926,18.630127,37.369354,0.052354,6.926999,103.971336,37.675526,0.029809,0.249617,0.108792,0.0,0.227003,2.079982,5.811303,2.178425,0.223836,3.379657,0.174453
min,0.000000,0.000000,-11.534750,0.000000,0.000000,0.000000,13.441751,0.000000,0.000000,0.000000,0.000000,2023.0,0.000000,0.000000,0.649994,3.200000,0.000000,-0.900024,0.000000
25%,9.068750,17.950000,2.447438,32.675001,0.000000,5.750000,89.981750,38.918751,0.000000,0.000000,0.000000,2023.0,0.578902,0.509375,6.699982,6.693750,0.587120,1.443741,0.000000
50%,121.874996,299.575012,19.800626,76.875000,0.000000,11.500000,180.960999,91.625000,0.000000,0.000000,0.000000,2023.0,0.791041,1.993750,11.625000,8.050000,0.791547,3.100006,0.000000
75%,343.337494,661.737488,38.861813,99.949997,0.000000,17.250000,271.244804,100.000000,0.000000,0.000000,0.100000,2023.0,0.868932,3.840625,15.025002,9.981250,0.870041,5.531242,0.000000
max,764.924988,834.950012,49.903500,100.000000,0.525000,23.000000,348.118500,100.000000,0.625000,1.000000,0.600000,2023.0,1.000000,9.062500,28.800018,13.675000,1.000000,18.174988,1.000000


In [7]:
# Fit the predictors
predictor_A = TabularPredictor(label=label, eval_metric='mean_absolute_error').fit(A, presets='good_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20231109_214419/"
Presets specified: ['good_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231109_214419/"
AutoGluon Version:  0.8.2
Python Version:     3.8.18
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 21.6.0: Wed Aug 10 14:28:23 PDT 2022; root:xnu-8020.141.5~2/RELEASE_ARM64_T6000
Disk Space Avail:   75.34 GB / 494.38 GB (15.2%)
Train Data Rows:    34085
Train Data Columns: 19
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (5733.42, 0.0, 630.59471, 1165.90242)
	If 'regression' is not the correct problem_type, please manually specify the problem_type paramet

[1000]	valid_set's l1: 179.585
[2000]	valid_set's l1: 175.416
[3000]	valid_set's l1: 173.309
[4000]	valid_set's l1: 172.013
[5000]	valid_set's l1: 171.009
[6000]	valid_set's l1: 170.328
[7000]	valid_set's l1: 169.677
[8000]	valid_set's l1: 169.129
[9000]	valid_set's l1: 168.603
[10000]	valid_set's l1: 168.233
[1000]	valid_set's l1: 182.254
[2000]	valid_set's l1: 177.796
[3000]	valid_set's l1: 174.219
[4000]	valid_set's l1: 172.802
[5000]	valid_set's l1: 171.764
[6000]	valid_set's l1: 170.855
[7000]	valid_set's l1: 170.043
[8000]	valid_set's l1: 169.441
[9000]	valid_set's l1: 168.74
[10000]	valid_set's l1: 168.332
[1000]	valid_set's l1: 177.892
[2000]	valid_set's l1: 173.518
[3000]	valid_set's l1: 170.69
[4000]	valid_set's l1: 168.852
[5000]	valid_set's l1: 167.948
[6000]	valid_set's l1: 167.232
[7000]	valid_set's l1: 166.55
[8000]	valid_set's l1: 166.078
[9000]	valid_set's l1: 165.838
[10000]	valid_set's l1: 165.658
[1000]	valid_set's l1: 168.336
[2000]	valid_set's l1: 163.887
[3000]	v

	-166.4203	 = Validation score   (-mean_absolute_error)
	5399.11s	 = Training   runtime
	6.13s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy


[1000]	valid_set's l1: 176.432
[2000]	valid_set's l1: 174.943
[3000]	valid_set's l1: 174.376
[4000]	valid_set's l1: 174.241
[1000]	valid_set's l1: 179.466
[2000]	valid_set's l1: 176.83
[3000]	valid_set's l1: 175.67
[4000]	valid_set's l1: 175.498
[5000]	valid_set's l1: 175.346
[1000]	valid_set's l1: 175.538
[2000]	valid_set's l1: 174.143
[3000]	valid_set's l1: 173.556
[4000]	valid_set's l1: 173.235
[5000]	valid_set's l1: 172.844
[6000]	valid_set's l1: 172.787
[7000]	valid_set's l1: 172.74
[8000]	valid_set's l1: 172.665
[9000]	valid_set's l1: 172.703
[10000]	valid_set's l1: 172.67
[1000]	valid_set's l1: 168.724
[2000]	valid_set's l1: 166.74
[1000]	valid_set's l1: 178.364
[2000]	valid_set's l1: 175.628
[3000]	valid_set's l1: 175.288
[4000]	valid_set's l1: 175.465
[1000]	valid_set's l1: 178.276
[2000]	valid_set's l1: 176.38
[3000]	valid_set's l1: 175.563
[4000]	valid_set's l1: 175.469
[1000]	valid_set's l1: 179.573
[2000]	valid_set's l1: 177.713
[3000]	valid_set's l1: 176.919
[4000]	valid_

	-174.3548	 = Validation score   (-mean_absolute_error)
	275.27s	 = Training   runtime
	1.8s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ...
	-185.2615	 = Validation score   (-mean_absolute_error)
	13.82s	 = Training   runtime
	0.59s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-182.3349	 = Validation score   (-mean_absolute_error)
	541.29s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: ExtraTreesMSE_BAG_L1 ...
	-186.4839	 = Validation score   (-mean_absolute_error)
	2.98s	 = Training   runtime
	0.71s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-191.7861	 = Validation score   (-mean_absolute_error)
	121.72s	 = Training   runtime
	0.17s	 = Validation runtime
Fitting model: XGBoost_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with Se

[1000]	valid_set's l1: 168.006
[2000]	valid_set's l1: 166.46
[3000]	valid_set's l1: 166.026
[4000]	valid_set's l1: 165.901
[5000]	valid_set's l1: 165.859
[6000]	valid_set's l1: 165.839
[7000]	valid_set's l1: 165.837
[8000]	valid_set's l1: 165.834
[9000]	valid_set's l1: 165.832
[10000]	valid_set's l1: 165.83
[1000]	valid_set's l1: 171.441
[2000]	valid_set's l1: 169.7
[3000]	valid_set's l1: 169.417
[4000]	valid_set's l1: 169.3
[5000]	valid_set's l1: 169.26
[6000]	valid_set's l1: 169.235
[7000]	valid_set's l1: 169.225
[8000]	valid_set's l1: 169.221
[9000]	valid_set's l1: 169.219
[10000]	valid_set's l1: 169.216
[1000]	valid_set's l1: 165.781
[2000]	valid_set's l1: 164.29
[3000]	valid_set's l1: 163.866
[4000]	valid_set's l1: 163.682
[5000]	valid_set's l1: 163.632
[6000]	valid_set's l1: 163.616
[7000]	valid_set's l1: 163.607
[8000]	valid_set's l1: 163.606
[9000]	valid_set's l1: 163.604
[10000]	valid_set's l1: 163.603
[1000]	valid_set's l1: 158.125
[2000]	valid_set's l1: 156.614
[3000]	valid_

	-166.7663	 = Validation score   (-mean_absolute_error)
	5209.08s	 = Training   runtime
	11.26s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-160.561	 = Validation score   (-mean_absolute_error)
	0.27s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting 9 L2 models ...
Fitting model: LightGBMXT_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy


[1000]	valid_set's l1: 169.33


	-162.954	 = Validation score   (-mean_absolute_error)
	39.14s	 = Training   runtime
	0.14s	 = Validation runtime
Fitting model: LightGBM_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-159.7898	 = Validation score   (-mean_absolute_error)
	17.53s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L2 ...
	-158.7246	 = Validation score   (-mean_absolute_error)
	23.05s	 = Training   runtime
	0.59s	 = Validation runtime
Fitting model: CatBoost_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-159.9892	 = Validation score   (-mean_absolute_error)
	14.67s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: ExtraTreesMSE_BAG_L2 ...
	-158.5223	 = Validation score   (-mean_absolute_error)
	2.97s	 = Training   runtime
	0.64s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with Sequ

In [8]:
predictor_B = TabularPredictor(label=label, eval_metric='mean_absolute_error').fit(B, presets='good_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20231110_011813/"
Presets specified: ['good_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231110_011813/"
AutoGluon Version:  0.8.2
Python Version:     3.8.18
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 21.6.0: Wed Aug 10 14:28:23 PDT 2022; root:xnu-8020.141.5~2/RELEASE_ARM64_T6000
Disk Space Avail:   73.54 GB / 494.38 GB (14.9%)
Train Data Rows:    32843
Train Data Columns: 19
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (1152.3, -0.0, 96.82773, 193.9487)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter 

[1000]	valid_set's l1: 31.8483
[2000]	valid_set's l1: 29.9723
[3000]	valid_set's l1: 28.9975
[4000]	valid_set's l1: 28.4245
[5000]	valid_set's l1: 27.941
[6000]	valid_set's l1: 27.5908
[7000]	valid_set's l1: 27.3134
[8000]	valid_set's l1: 27.1214
[9000]	valid_set's l1: 26.9449
[10000]	valid_set's l1: 26.7864
[1000]	valid_set's l1: 31.6322
[2000]	valid_set's l1: 29.8926
[3000]	valid_set's l1: 29.0257
[4000]	valid_set's l1: 28.386
[5000]	valid_set's l1: 27.9288
[6000]	valid_set's l1: 27.6033
[7000]	valid_set's l1: 27.3299
[8000]	valid_set's l1: 27.129
[9000]	valid_set's l1: 26.9103
[10000]	valid_set's l1: 26.7573
[1000]	valid_set's l1: 32.9894
[2000]	valid_set's l1: 31.2049
[3000]	valid_set's l1: 30.1804
[4000]	valid_set's l1: 29.5329
[5000]	valid_set's l1: 29.0936
[6000]	valid_set's l1: 28.7607
[7000]	valid_set's l1: 28.4592
[8000]	valid_set's l1: 28.195
[9000]	valid_set's l1: 28.0323
[10000]	valid_set's l1: 27.8674
[1000]	valid_set's l1: 31.0725
[2000]	valid_set's l1: 29.3381
[3000]	va

	-26.7032	 = Validation score   (-mean_absolute_error)
	550.67s	 = Training   runtime
	7.1s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy


[1000]	valid_set's l1: 29.065
[2000]	valid_set's l1: 28.1471
[3000]	valid_set's l1: 27.7272
[4000]	valid_set's l1: 27.5554
[5000]	valid_set's l1: 27.3885
[6000]	valid_set's l1: 27.2734
[7000]	valid_set's l1: 27.1878
[8000]	valid_set's l1: 27.1459
[9000]	valid_set's l1: 27.1103
[10000]	valid_set's l1: 27.093
[1000]	valid_set's l1: 29.5963
[2000]	valid_set's l1: 28.4423
[3000]	valid_set's l1: 27.9248
[4000]	valid_set's l1: 27.6001
[5000]	valid_set's l1: 27.4143
[6000]	valid_set's l1: 27.3002
[7000]	valid_set's l1: 27.2417
[8000]	valid_set's l1: 27.1893
[9000]	valid_set's l1: 27.1625
[10000]	valid_set's l1: 27.1302
[1000]	valid_set's l1: 30.7986
[2000]	valid_set's l1: 29.539
[3000]	valid_set's l1: 29.0113
[4000]	valid_set's l1: 28.7352
[5000]	valid_set's l1: 28.5577
[6000]	valid_set's l1: 28.4435
[7000]	valid_set's l1: 28.3349
[8000]	valid_set's l1: 28.2648
[9000]	valid_set's l1: 28.2187
[10000]	valid_set's l1: 28.1876
[1000]	valid_set's l1: 28.9981
[2000]	valid_set's l1: 27.7921
[3000]	v

	-27.0623	 = Validation score   (-mean_absolute_error)
	533.13s	 = Training   runtime
	7.18s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ...
	-28.996	 = Validation score   (-mean_absolute_error)
	16.62s	 = Training   runtime
	0.6s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-28.7479	 = Validation score   (-mean_absolute_error)
	574.88s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: ExtraTreesMSE_BAG_L1 ...
	-30.1387	 = Validation score   (-mean_absolute_error)
	3.02s	 = Training   runtime
	0.72s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-32.3621	 = Validation score   (-mean_absolute_error)
	110.43s	 = Training   runtime
	0.15s	 = Validation runtime
Fitting model: XGBoost_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with Sequenti

[1000]	valid_set's l1: 26.1245
[2000]	valid_set's l1: 25.3532
[3000]	valid_set's l1: 25.1194
[4000]	valid_set's l1: 25.049
[5000]	valid_set's l1: 25.0179
[6000]	valid_set's l1: 25.003
[7000]	valid_set's l1: 24.9972
[8000]	valid_set's l1: 24.995
[9000]	valid_set's l1: 24.9935
[10000]	valid_set's l1: 24.9924
[1000]	valid_set's l1: 26.1241
[2000]	valid_set's l1: 25.372
[3000]	valid_set's l1: 25.1414
[4000]	valid_set's l1: 25.0752
[5000]	valid_set's l1: 25.05
[6000]	valid_set's l1: 25.0322
[7000]	valid_set's l1: 25.0244
[8000]	valid_set's l1: 25.0209
[9000]	valid_set's l1: 25.0185
[10000]	valid_set's l1: 25.0176
[1000]	valid_set's l1: 27.1436
[2000]	valid_set's l1: 26.2773
[3000]	valid_set's l1: 26.0313
[4000]	valid_set's l1: 25.9425
[5000]	valid_set's l1: 25.9138
[6000]	valid_set's l1: 25.8947
[7000]	valid_set's l1: 25.8855
[8000]	valid_set's l1: 25.8817
[9000]	valid_set's l1: 25.8804
[10000]	valid_set's l1: 25.8794
[1000]	valid_set's l1: 25.8939
[2000]	valid_set's l1: 25.1313
[3000]	vali

	-24.8677	 = Validation score   (-mean_absolute_error)
	1964.94s	 = Training   runtime
	10.6s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-24.538	 = Validation score   (-mean_absolute_error)
	0.37s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting 9 L2 models ...
Fitting model: LightGBMXT_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-22.9165	 = Validation score   (-mean_absolute_error)
	39.66s	 = Training   runtime
	0.19s	 = Validation runtime
Fitting model: LightGBM_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-22.8362	 = Validation score   (-mean_absolute_error)
	22.37s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L2 ...
	-21.9123	 = Validation score   (-mean_absolute_error)
	29.84s	 = Training   runtime
	0.58s	 = Validation runtime
Fitting model: CatBoost_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fit

[1000]	valid_set's l1: 22.107
[2000]	valid_set's l1: 22.0451
[3000]	valid_set's l1: 22.0388
[4000]	valid_set's l1: 22.0311
[5000]	valid_set's l1: 22.0306
[6000]	valid_set's l1: 22.0292
[7000]	valid_set's l1: 22.0287
[8000]	valid_set's l1: 22.0286
[9000]	valid_set's l1: 22.0286
[10000]	valid_set's l1: 22.0286


	-22.4548	 = Validation score   (-mean_absolute_error)
	372.08s	 = Training   runtime
	1.55s	 = Validation runtime
Fitting model: WeightedEnsemble_L3 ...
	-21.6537	 = Validation score   (-mean_absolute_error)
	0.31s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 6872.42s ... Best model: "WeightedEnsemble_L3"
Automatically performing refit_full as a post-fit operation (due to `.fit(..., refit_full=True)`
Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix "_FULL" and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `predictor.fit` call.
	To learn more, refer to the `.refit_full` method docstring which explains how "_FULL" models differ from normal models.
Fitting 1 L1 models ...
Fitting model: LightGBMXT_BAG_L1_FULL ...
	894.72s	 = Training   runtime
Fitting 1 L1 models ...
Fittin

In [9]:
predictor_C = TabularPredictor(label=label, eval_metric='mean_absolute_error').fit(C, presets='good_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20231110_035354/"
Presets specified: ['good_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231110_035354/"
AutoGluon Version:  0.8.2
Python Version:     3.8.18
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 21.6.0: Wed Aug 10 14:28:23 PDT 2022; root:xnu-8020.141.5~2/RELEASE_ARM64_T6000
Disk Space Avail:   72.84 GB / 494.38 GB (14.7%)
Train Data Rows:    26095
Train Data Columns: 19
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and label-values can't be converted to int).
	Label info (max, min, mean, stddev): (999.6, -0.0, 77.63106, 165.81688)
	If 'regression' is not the correct problem_type, please manually specify the problem_type param

[1000]	valid_set's l1: 19.2761
[2000]	valid_set's l1: 18.6473
[3000]	valid_set's l1: 18.273
[4000]	valid_set's l1: 18.0101
[5000]	valid_set's l1: 17.8761
[6000]	valid_set's l1: 17.7816
[7000]	valid_set's l1: 17.7228
[8000]	valid_set's l1: 17.6732
[9000]	valid_set's l1: 17.6296
[10000]	valid_set's l1: 17.6092
[1000]	valid_set's l1: 20.8363
[2000]	valid_set's l1: 20.0447
[3000]	valid_set's l1: 19.6867
[4000]	valid_set's l1: 19.4437
[5000]	valid_set's l1: 19.2754
[6000]	valid_set's l1: 19.1927
[7000]	valid_set's l1: 19.1107
[8000]	valid_set's l1: 19.0342
[9000]	valid_set's l1: 18.9964
[10000]	valid_set's l1: 18.9444
[1000]	valid_set's l1: 19.4069
[2000]	valid_set's l1: 18.9106
[3000]	valid_set's l1: 18.5819
[4000]	valid_set's l1: 18.3649
[5000]	valid_set's l1: 18.1716
[6000]	valid_set's l1: 18.0846
[7000]	valid_set's l1: 17.9775
[8000]	valid_set's l1: 17.9146
[9000]	valid_set's l1: 17.8746
[10000]	valid_set's l1: 17.8328
[1000]	valid_set's l1: 20.011
[2000]	valid_set's l1: 19.4699
[3000]	

	-18.0893	 = Validation score   (-mean_absolute_error)
	551.64s	 = Training   runtime
	4.83s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy


[1000]	valid_set's l1: 19.029
[2000]	valid_set's l1: 18.6201
[3000]	valid_set's l1: 18.5777
[4000]	valid_set's l1: 18.5447
[5000]	valid_set's l1: 18.5339
[6000]	valid_set's l1: 18.5217
[7000]	valid_set's l1: 18.523
[8000]	valid_set's l1: 18.5147
[9000]	valid_set's l1: 18.5157
[10000]	valid_set's l1: 18.5139
[1000]	valid_set's l1: 20.3982
[2000]	valid_set's l1: 20.0788
[3000]	valid_set's l1: 19.9627
[4000]	valid_set's l1: 19.9222
[5000]	valid_set's l1: 19.8953
[6000]	valid_set's l1: 19.8765
[7000]	valid_set's l1: 19.8671
[8000]	valid_set's l1: 19.8631
[9000]	valid_set's l1: 19.8557
[10000]	valid_set's l1: 19.8545
[1000]	valid_set's l1: 19.862
[2000]	valid_set's l1: 19.5371
[3000]	valid_set's l1: 19.4585
[4000]	valid_set's l1: 19.3822
[5000]	valid_set's l1: 19.3325
[6000]	valid_set's l1: 19.3034
[7000]	valid_set's l1: 19.2913
[8000]	valid_set's l1: 19.2831
[9000]	valid_set's l1: 19.2786
[10000]	valid_set's l1: 19.2746
[1000]	valid_set's l1: 20.2371
[2000]	valid_set's l1: 19.9264
[3000]	v

	-18.9971	 = Validation score   (-mean_absolute_error)
	437.42s	 = Training   runtime
	3.58s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ...
	-20.4523	 = Validation score   (-mean_absolute_error)
	9.52s	 = Training   runtime
	0.36s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-19.5103	 = Validation score   (-mean_absolute_error)
	505.09s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: ExtraTreesMSE_BAG_L1 ...
	-20.3935	 = Validation score   (-mean_absolute_error)
	1.97s	 = Training   runtime
	0.49s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-21.3337	 = Validation score   (-mean_absolute_error)
	88.12s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: XGBoost_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with Sequenti

[1000]	valid_set's l1: 18.1003
[2000]	valid_set's l1: 17.9353
[3000]	valid_set's l1: 17.9034
[4000]	valid_set's l1: 17.8943
[5000]	valid_set's l1: 17.8907
[6000]	valid_set's l1: 17.8895
[7000]	valid_set's l1: 17.889
[8000]	valid_set's l1: 17.8887
[9000]	valid_set's l1: 17.8887
[10000]	valid_set's l1: 17.8886
[1000]	valid_set's l1: 19.0561
[2000]	valid_set's l1: 18.858
[3000]	valid_set's l1: 18.8336
[4000]	valid_set's l1: 18.8265
[5000]	valid_set's l1: 18.823
[6000]	valid_set's l1: 18.8223
[7000]	valid_set's l1: 18.8218
[8000]	valid_set's l1: 18.8217
[9000]	valid_set's l1: 18.8216
[10000]	valid_set's l1: 18.8217
[1000]	valid_set's l1: 18.4246
[2000]	valid_set's l1: 18.2573
[3000]	valid_set's l1: 18.23
[4000]	valid_set's l1: 18.2215
[5000]	valid_set's l1: 18.2179
[6000]	valid_set's l1: 18.2166
[7000]	valid_set's l1: 18.2163
[8000]	valid_set's l1: 18.2163
[9000]	valid_set's l1: 18.2162
[10000]	valid_set's l1: 18.2162
[1000]	valid_set's l1: 19.2838
[2000]	valid_set's l1: 19.1781
[3000]	val

	-18.2435	 = Validation score   (-mean_absolute_error)
	4876.69s	 = Training   runtime
	7.44s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-17.2385	 = Validation score   (-mean_absolute_error)
	0.22s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting 9 L2 models ...
Fitting model: LightGBMXT_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-17.574	 = Validation score   (-mean_absolute_error)
	31.19s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: LightGBM_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-17.5272	 = Validation score   (-mean_absolute_error)
	18.94s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L2 ...
	-17.1669	 = Validation score   (-mean_absolute_error)
	16.0s	 = Training   runtime
	0.35s	 = Validation runtime
Fitting model: CatBoost_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitti

In [10]:
# predictor_A.evaluate(test_A, silent=True)
# predictor_B.evaluate(B, silent=True)
# predictor_C.evaluate(C, silent=True)

In [11]:
# predictor_A.leaderboard(test_A, silent=False)
# predictor_B.leaderboard(B, silent=True)
# predictor_C.leaderboard(C, silent=True)

predictor_A.plot_ensemble_model()
predictor_B.plot_ensemble_model()
predictor_C.plot_ensemble_model()

'AutogluonModels/ag-20231110_035354/ensemble_model.png'

In [12]:
# Loading best predictors
# predictor_A = TabularPredictor.load("AutogluonModels/ag-20231106_113214/")
# predictor_B = TabularPredictor.load("AutogluonModels/ag-20231106_114820/")
# predictor_C = TabularPredictor.load("AutogluonModels/ag-20231106_120433/")

In [13]:
test_A = TabularDataset(test_A)
test_B = TabularDataset(test_B)
test_C = TabularDataset(test_C)

test_A.describe()

,total_radiation,clear_sky_rad:W,sun_elevation:d,effective_cloud_cover:p,rain_water:kgm2,hour,sun_azimuth:d,total_cloud_cover:p,snow_accumulation,binned_month,super_cooled_liquid_water:kgm2,year,sfc_pressure:hPa_scaled,average_wind_speed,t_1000hPa:C,absolute_humidity_2m:gm3,msl_pressure:hPa_scaled,temp_dewpoint_diff,dew_or_rime:idx
count,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.0,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000
mean,199.684021,341.014008,20.294441,64.132294,0.013333,11.500000,179.181381,69.309792,0.001563,0.066667,0.063924,2023.0,0.704526,2.407205,11.573374,8.205903,0.707816,3.923195,0.034722
std,215.015656,306.911926,18.630127,37.369354,0.052354,6.926999,103.971336,37.675526,0.029809,0.249617,0.108792,0.0,0.227003,2.079982,5.811303,2.178425,0.223836,3.379657,0.174453
min,0.000000,0.000000,-11.534750,0.000000,0.000000,0.000000,13.441751,0.000000,0.000000,0.000000,0.000000,2023.0,0.000000,0.000000,0.649994,3.200000,0.000000,-0.900024,0.000000
25%,9.068750,17.950000,2.447438,32.675001,0.000000,5.750000,89.981750,38.918751,0.000000,0.000000,0.000000,2023.0,0.578902,0.509375,6.699982,6.693750,0.587120,1.443741,0.000000
50%,121.874996,299.575012,19.800626,76.875000,0.000000,11.500000,180.960999,91.625000,0.000000,0.000000,0.000000,2023.0,0.791041,1.993750,11.625000,8.050000,0.791547,3.100006,0.000000
75%,343.337494,661.737488,38.861813,99.949997,0.000000,17.250000,271.244804,100.000000,0.000000,0.000000,0.100000,2023.0,0.868932,3.840625,15.025002,9.981250,0.870041,5.531242,0.000000
max,764.924988,834.950012,49.903500,100.000000,0.525000,23.000000,348.118500,100.000000,0.625000,1.000000,0.600000,2023.0,1.000000,9.062500,28.800018,13.675000,1.000000,18.174988,1.000000


In [14]:
# Create submission

output_file = 'autogluon_submission_new.csv'

pred_A = predictor_A.predict(test_A)
pred_B = predictor_B.predict(test_B)
pred_C = predictor_C.predict(test_C)

pred_A = np.clip(pred_A, 0, None)
pred_B = np.clip(pred_B, 0, None)
pred_C = np.clip(pred_C, 0, None)

# Concatenate predictions
predictions = np.concatenate([pred_A, pred_B, pred_C])

# Create an id array
ids = np.arange(0, len(predictions))

# Create a DataFrame
df = pd.DataFrame({
    'id': ids,
    'prediction': predictions
})

# Save to CSV
df.to_csv(output_file, index=False)
print(f"Submission saved to {output_file}")

Submission saved to autogluon_submission_new.csv


In [15]:
# For testing
pred_A = predictor_A.predict(X_A)
pred_B = predictor_B.predict(X_B)
pred_C = predictor_C.predict(X_C)

# Evaluate 
mae_A = mean_absolute_error(y_A, pred_A)
mae_B = mean_absolute_error(y_B, pred_B)
mae_C = mean_absolute_error(y_C, pred_C)

print(f"MAE A: {mae_A}")
print(f"MAE B: {mae_B}")
print(f"MAE C: {mae_C}")

MAE A: 44.29916398310762
MAE B: 6.006177026233198
MAE C: 5.991450980870519
